In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from math import log
from collections import Counter

In [2]:
datasets = [['青年', '否', '否', '一般', '否'],
            ['青年', '否', '否', '好', '否'],
            ['青年', '是', '否', '好', '是'],
            ['青年', '是', '是', '一般', '是'],
            ['青年', '否', '否', '一般', '否'],
            ['中年', '否', '否', '一般', '否'],
            ['中年', '否', '否', '好', '否'],
            ['中年', '是', '是', '好', '是'],
            ['中年', '否', '是', '非常好', '是'],
            ['中年', '否', '是', '非常好', '是'],
            ['老年', '否', '是', '非常好', '是'],
            ['老年', '否', '是', '好', '是'],
            ['老年', '是', '否', '好', '是'],
            ['老年', '是', '否', '非常好', '是'],
            ['老年', '否', '否', '一般', '否'],
            ]
labels = [u'年龄',u'有工作',u'有自己的房子',u'信贷情况',u'类别']

In [3]:
train_data = pd.DataFrame(datasets,columns=labels)

In [4]:
train_data

,年龄,有工作,有自己的房子,信贷情况,类别
0,青年,否,否,一般,否
1,青年,否,否,好,否
2,青年,是,否,好,是
3,青年,是,是,一般,是
4,青年,否,否,一般,否
5,中年,否,否,一般,否
6,中年,否,否,好,否
7,中年,是,是,好,是
8,中年,否,是,非常好,是
9,中年,否,是,非常好,是


In [8]:
def calc_ent(datasets):
    """
        input:data frame
        output:计算label的信息熵
    """
    data_length = len(datasets)
    label_count = {}
    for i in range(data_length):
        label = datasets[i][-1]
        if label not in label_count:
            label_count[label] = 0
        label_count[label] += 1
        
    # 信息熵的公式
    ent = -sum([(p / data_length) * log(p / data_length, 2)
                for p in label_count.values()])
    return ent

In [12]:
# 信息增益
def info_gain(ent, cond_ent): #info_gain_ratio
    return ent - cond_ent     #    return ent - cond_ent/ivent

In [13]:
def info_gain_train(datasets):
    """
    遍历所有特征，选择增益最大的特征
    """
    count = len(datasets[0]) - 1
    ent = calc_ent(datasets)
    best_feature = []
    for c in range(count):
        c_info_gain = info_gain(ent, cond_ent(datasets, axis=c))
        best_feature.append((c, c_info_gain))
        #[[年龄，a],[有房子，b]]
        print('特征({}) - info_gain - {:.3f}'.format(labels[c], c_info_gain))
    best_ = max(best_feature, key=lambda x: x[-1])
    return '特征({})的信息增益最大，选择为根节点特征'.format(labels[best_[0]])

In [17]:
class Node:
    def __init__(self,root=True,label=None,feature_name=None,feature=None):
        self.root = root
        self.label = label
        self.feature_name = feature_name
        self.feature = feature
        self.tree = {}
        self.result = {'label':self.label,
                       'feature':self.feature,
                       'tree':self.tree}
    def __repr__(self):
        return '{}'.format(self.result)
    
    def add_node(self,val,node):
        self.tree[val] = node
    
    def predict(self,features):
        if self.root is True:
            return self.label
        return self.tree[features[self.feature]].predict(features)
    
class DTree:
    def __init__(self,epsilon=0.1):
        self.epsilon = epsilon
        self._tree = {}
        
    @staticmethod
    def calc_ent(datasets):
        data_length = len(datasets)
        label_count = {}
        for i in range(data_length):
            label = datasets[i][-1]
            if label not in label_count:
                label_count[label] = 0
            label_count[label] += 1
        ent = -sum([(p / data_length) * log(p / data_length, 2)
                    for p in label_count.values()])
        return ent
    
    def cond_ent(self,datasets,axis=0):
        data_length = len(datasets)
        feature_sets = {}
        for i in range(data_length):
            feature = datasets[i][axis]
            if feature not in feature_sets:
                feature_sets[feature] = []
            feature_sets[feature].append(datasets[i])
        cond_end = sum([(len(p)/data_length) * calc_ent(p) for p in feature_sets.values()])
        return cond_end
    
    def info_gain(ent,cond_ent):
        return ent - cond_ent    ##信息增益率： (ent - cond_ent) / iv_ent
    
    def info_gain_train(self,datasets):
        count = len(datasets[0]) - 1
        ent = calc_ent(datasets)
        best_feature = []
        for c in range(count):
            c_info_gain = info_gain(ent,self.cond_ent(datasets,axis=c))
            best_feature.append((c,c_info_gain))
        best_ = max(best_feature, key = lambda x: x[-1])
        return best_
    
    def train(self,train_data):
        x_train,y_train,features = train_data.iloc[:,:-1],train_data.iloc[:,-1],train_data.columns[:-1]
        
        if len(y_train.value_counts()) == 1:
            return Node(root=True,label=y_train.iloc[0])
        
        if len(features) == 0:
            return Node(root=True,label=y_train.value_counts().sort_values(ascending=False).index[0])
        
        max_feature,max_info_gain = self.info_gain_train(np.array(train_data))
        max_feature_name = features[max_feature]
        if max_info_gain < self.epsilon:
            return Node(root=True,label=y_train.value_counts().sort_values(ascending=False).index[0])
        
        node_tree = Node(root=False,feature_name=max_feature_name,feature=max_feature)
        
        feature_list = train_data[max_feature_name].value_counts().index
        for f in feature_list:
            sub_train_df = train_data.loc[train_data[max_feature_name] == f].drop([max_feature_name],axis=1)
            
            sub_tree = self.train(sub_train_df)
            node_tree.add_node(f,sub_tree)
            
        return node_tree
    
    def fit(self,train_data):
        self._tree = self.train(train_data)
        return self._tree
    
    def predict(self,X_test):
        return self._tree.predict(X_test)

In [18]:
data_df = pd.DataFrame(datasets,columns = labels)

In [19]:
dt = DTree()
tree = dt.fit(data_df)

In [20]:
tree

{'label': None, 'feature': 2, 'tree': {'否': {'label': None, 'feature': 1, 'tree': {'否': {'label': '否', 'feature': None, 'tree': {}}, '是': {'label': '是', 'feature': None, 'tree': {}}}}, '是': {'label': '是', 'feature': None, 'tree': {}}}}